### Result Inspection
This notebook provides an example on how to evaluate trained models by generating trajectories, visualizing the dynamics and evaluating metrics.

In [ ]:
import numpy as np
import torch as tc
import matplotlib.pyplot as plt
import seaborn as sns
from bptt.models import Model

In [ ]:
data_path = "" # e.g. "Experiments/Table1/Lorenz63/data/lorenz63_test.npy"
model_path = "" # e.g. "results/Lorenz63/M22B20tau25T200"

In [ ]:
# load data
lor63 = tc.tensor(np.load(data_path))

In [ ]:
# restore model checkpoint
m = Model()
m.init_from_model_path(model_path, epoch=5000)
m.eval()
print(repr(m), f"\nNumber of Parameters: {m.get_num_trainable()}")

In [ ]:
# generate a trajectory freely running the model
T = len(lor63)
X, Z = m.generate_free_trajectory(lor63, T)
X.shape, Z.shape

In [ ]:
# some plotting
plt.figure(figsize=(12, 8))
plt.subplot(211)
plt.plot(X[:1000])
plt.ylabel("X")
plt.subplot(212)
plt.plot(Z[:1000, 3:])
plt.ylabel("Z")
plt.xlabel("T")
plt.show()

In [ ]:
# plot AW matrix
A, W, h = m.latent_model.get_latent_parameters()
AW = tc.diag(A.detach()) + W.detach()
plt.figure(figsize=(12, 8))
sns.heatmap(data=AW, annot=True, cmap="Oranges_r", square=True, fmt='.2f',
            yticklabels=False, xticklabels=False)
plt.show()

In [ ]:
# plot state space trajectories
x_np = Z.numpy()[:10000]
lor_np = lor63.numpy()[:10000]
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection='3d')
ax.plot3D(lor_np[:, 0], lor_np[:, 1], lor_np[:, 2], label="GT")
ax.plot3D(x_np[:, 0], x_np[:, 1], x_np[:, 2], label="Generated")
plt.legend(fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
# compute KLx and PSE
from evaluation.klx import klx_metric
from evaluation.pse import power_spectrum_error_per_dim

klx = klx_metric(X, lor63, n_bins=30) # evaluated on one long trajectory here!
psc = np.mean(power_spectrum_error_per_dim(X.unsqueeze(0), lor63.unsqueeze(0), 20, 20000))

print(f"KLx = {klx:.3f}, PSC = {psc:.3f}")